# CS5228 Project: HDB Resale Price Prediction

<a href="https://colab.research.google.com/github/Biscuitkru/CS5228_Project/blob/main/CS5228_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Overview
This notebook contains the complete analysis and modeling pipeline for predicting HDB (Housing & Development Board) resale prices in Singapore. The project implements various machine learning techniques including:
- Exploratory Data Analysis (EDA)
- Feature Engineering with proximity-based features
- Multiple regression models (Linear, Polynomial, Tree-based)
- Advanced gradient boosting methods (XGBoost)

**Authors:** CS5228 Group 34  
**Course:** Knowledge Discovery and Data Mining  
**Semester:** AY2025/26

In [ ]:
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q xgboost
!pip install -q tqdm
!pip install -q joblib


In [ ]:
import os, re, math, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import math
import json

from pathlib import Path
from datetime import datetime
from typing import Optional, Tuple, Dict, List, Any
from collections import Counter

# for modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error

from tqdm.auto import tqdm
import joblib

In [ ]:
# # ########################################
# # # Configs for kaggle run (upload the data folder as a zip)
# # ########################################

# # Main
# train_path = "/kaggle/input/cs5228-dataset-g34/data/train.csv"
# test_path = "/kaggle/input/cs5228-dataset-g34/data/test.csv"

# # Auxiliary data
# hdb_data_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-hdb-block-details.csv"
# gov_hawkers_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-gov-hawkers.csv"
# mrt_stations_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-mrt-stations.csv"
# pri_schools_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-primary-schools.csv"
# sec_schools_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-secondary-schools.csv"
# shopping_malls_path = "/kaggle/input/cs5228-dataset-g34/data/auxiliary-data/sg-shopping-malls.csv"

# # Modeling controls
# seed = 777
# rng = np.random.default_rng(seed)

In [ ]:
# ########################################
# # Configs for colab
# ########################################

# from google.colab import drive
# drive.mount('/content/drive')

# # Main
# train_path = "/content/drive/MyDrive/CS Masters/CS5228/train.csv"
# test_path = "/content/drive/MyDrive/CS Masters/CS5228/test.csv"

# # Auxiliary data
# hdb_data_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-hdb-block-details.csv"
# gov_hawkers_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-gov-hawkers.csv"
# mrt_stations_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-mrt-stations.csv"
# pri_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-primary-schools.csv"
# sec_schools_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-secondary-schools.csv"
# shopping_malls_path = "/content/drive/MyDrive/CS Masters/CS5228/auxiliary-data/sg-shopping-malls.csv"

# # Modeling controls
# seed = 777
# rng = np.random.default_rng(seed)

In [ ]:
###############################
# Configs for local run
###############################

local_path = r"C:/Users/GoCh661/Downloads/cs5228/data"


# Main
train_path = local_path + "/train.csv"
test_path = local_path + "/test.csv"

# Auxiliary data
hdb_data_path = local_path + "/auxiliary-data/sg-hdb-block-details.csv"
gov_hawkers_path = local_path + "/auxiliary-data/sg-gov-hawkers.csv"
mrt_stations_path = local_path + "/auxiliary-data/sg-mrt-stations.csv"
pri_schools_path = local_path + "/auxiliary-data/sg-primary-schools.csv"
sec_schools_path = local_path + "/auxiliary-data/sg-secondary-schools.csv"
shopping_malls_path = local_path + "/auxiliary-data/sg-shopping-malls.csv"

# Modeling controls
seed = 777
np.random.seed(seed)

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Standardize column names to lowercase
train_df.columns = train_df.columns.str.lower()
test_df.columns = test_df.columns.str.lower()

print("Column names standardized to lowercase")
print(f"Train columns: {list(train_df.columns)[:10]}...")  # Show first 10

In [ ]:
# Verify data loaded correctly
print("="*70)
print("DATA LOADING VERIFICATION")
print("="*70)
print(f"Training data loaded from: {train_path}")
print(f"Test data loaded from: {test_path}")
print(f"\nTraining set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"\nSample columns: {list(train_df.columns)[:5]}")
print("\nFirst few rows of training data:")
train_df.head(3)

---
## Exploratory Data Analysis (EDA) and Data Preprocessing

This section provides comprehensive analysis of the HDB resale dataset, including:
- Dataset overview and statistical summaries
- Target variable distribution analysis
- Feature correlation studies
- Categorical and temporal pattern exploration
- Data quality assessment

The findings from this EDA guide our feature engineering and model selection strategies.

### 1. Dataset Overview and Basic Statistics

**Purpose:** Examine the basic structure, dimensions, and statistical properties of the training dataset.

The following analysis provides:
- Descriptive statistics for all numerical features
- Data type distribution
- Missing value assessment
- Initial data quality checks

In [ ]:
train_df.describe() # No NA values at all

In [ ]:
# Date Manipulation
def process_month_column(df):
    df['month'] = pd.to_datetime(df['month'], format="%Y-%m")
    df['sale_year'] = df['month'].dt.year
    df['sale_month'] = df['month'].dt.month
    # sale_quarter as a feature to capture potential seasonality in resale prices.
    df['sale_quarter'] = df['month'].dt.quarter
    return df

train_df = process_month_column(train_df)
test_df = process_month_column(test_df)

# additional month_index to capture trend
abs_month_train = train_df["sale_year"]*12 + train_df["sale_month"]
abs_month_test  = test_df["sale_year"]*12 + test_df["sale_month"]
train_df["month_index"] = (abs_month_train - abs_month_train.min()).astype(float)
test_df["month_index"]  = (abs_month_test  - abs_month_train.min()).astype(float)

In [ ]:
# To remove trailing whitespace and reduce multiple spaces to a single space
def norm_text_basic(s):
    if not isinstance(s, str):
        return s
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    return s

# Normalise + uppercase, mainly for the str values
def norm_key(s, case="upper"):
    s = norm_text_basic(s)
    if isinstance(s, str):
        return s.upper() if case == "upper" else s.casefold()
    return s

In [ ]:
for col in ["town","flat_type", "block", "street", "flat_model"]:
    if col in train_df.columns: train_df[col] = train_df[col].map(norm_key)
    if col in test_df.columns: test_df[col]  = test_df[col].map(norm_key)

In [ ]:
# Standardize flat type
def standardise_flat_type(val: str) -> str:
    v = val.strip().replace("_", " ").replace("-", " ").replace("  ", " ")
    v = v.replace("ROOM", " ROOM")
    v = re.sub(r"\s+", " ", v).strip()
    return v

In [ ]:
train_df['flat_type'] = train_df['flat_type'].apply(standardise_flat_type)
test_df['flat_type'] = test_df['flat_type'].apply(standardise_flat_type)
print(train_df['flat_type'].unique())

In [ ]:
# General Price hierarchy for HDB flats
flat_type_order = ["1 ROOM","2 ROOM","3 ROOM","4 ROOM","5 ROOM","MULTI GENERATION","EXECUTIVE"]
order_map = {ft:i for i, ft in enumerate(flat_type_order)}

train_df["flat_type_price_hier"] = train_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))
test_df["flat_type_price_hier"]  = test_df["flat_type"].map(lambda x: order_map.get(str(x).upper(), np.nan))

# For Decision trees
for col, val in [("is_exec","EXECUTIVE"), ("is_multigen","MULTI GENERATION")]:
    train_df[col] = (train_df["flat_type"].str.upper() == val).astype(int)
    test_df[col]  = (test_df["flat_type"].str.upper() == val).astype(int)

In [ ]:
# No. of Bedrooms
BEDROOMS_MAP = {"1 ROOM":0,"2 ROOM":1,"3 ROOM":2,"4 ROOM":3,"5 ROOM":4,"EXECUTIVE":3,"MULTI GENERATION":4}
train_df["bedrooms_est"] = train_df["flat_type"].str.upper().map(BEDROOMS_MAP)
test_df["bedrooms_est"]  = test_df["flat_type"].str.upper().map(BEDROOMS_MAP)

In [ ]:
# Floor Range, just going to set as the middle numnber
def parse_floor_mid(floor_range) -> float:
    if isinstance(floor_range, (float)):
        return float(floor_range)
    m = re.findall(r"\d{1,2}", floor_range)
    if len(m) >= 2:
        lo, hi = int(m[0]), int(m[1])
        if lo <= hi:
            return (lo + hi) / 2.0
    return np.nan

In [ ]:
train_df['floor_range'] = train_df['floor_range'].apply(parse_floor_mid).astype(float)
train_df = train_df.rename(columns={"floor_range": "floor_mid"})

test_df['floor_range'] = test_df['floor_range'].apply(parse_floor_mid).astype(float)
test_df = test_df.rename(columns={"floor_range": "floor_mid"})

print(np.sort(train_df['floor_mid'].unique()))
print(np.sort(test_df['floor_mid'].unique()))

In [ ]:
print(train_df['eco_category'].unique())
print(test_df['eco_category'].unique())

# Current dataset is taken directly from the Kaggle scoreboard which is missing eco_category (Might need to double check whether this is true)
train_df = train_df.drop('eco_category', axis=1)
test_df = test_df.drop('eco_category', axis=1)

In [ ]:
# Flat age, years left from 99
train_df = train_df.rename(columns={"lease_commence_data": "lease_commence_date"})
test_df = test_df.rename(columns={"lease_commence_data": "lease_commence_date"})

train_df["lease_left"] = 99 - (train_df["sale_year"] - train_df["lease_commence_date"])
test_df["lease_left"] = 99 - (test_df["sale_year"] - test_df["lease_commence_date"])

In [ ]:
# HDB price drops faster as the remaining lease falls—use non-linear transforms and an approximate leasehold-relativity curve

for df in (train_df, test_df):
    df["lease_left_sq"]  = df["lease_left"]**2
    df["lease_left_log"] = np.log1p(df["lease_left"].clip(lower=0))

    # Bala-style approximation using anchor points (convex decay)
    anchors_years = np.array([30, 60, 99], dtype=float)
    anchors_rel   = np.array([0.60, 0.80, 0.96], dtype=float)
    df["lease_rel_approx"] = np.interp(df["lease_left"].clip(0, 99), anchors_years, anchors_rel)

In [ ]:
# Loading Aux Data
aux_dict = {
    "hdb": pd.read_csv(hdb_data_path),
    "mrt": pd.read_csv(mrt_stations_path),
    "pri": pd.read_csv(pri_schools_path),
    "sec": pd.read_csv(sec_schools_path),
    "malls": pd.read_csv(shopping_malls_path),
    "hawkers": pd.read_csv(gov_hawkers_path)
}

In [ ]:
# HDB Data
hdb = aux_dict["hdb"].copy()

hdb.columns = hdb.columns.str.lower()

for col in ["town","block", "address", "subzone", "planning_area", "region"]:
    if col in hdb.columns: hdb[col] = hdb[col].map(norm_key)

In [ ]:
hdb_pairs = hdb[["town","block"]].drop_duplicates()
train_pairs = train_df[["town","block"]]
test_pairs  = test_df[["town","block"]]

In [ ]:
train_cov = train_pairs.merge(hdb_pairs, on=["town","block"], how="left", indicator=True)
test_cov = test_pairs.merge(hdb_pairs,  on=["town","block"], how="left", indicator=True)

train_match_n = (train_cov["_merge"] == "both").sum()
test_match_n = (test_cov["_merge"] == "both").sum()

print(f"[COVERAGE] Train matches: {train_match_n}/{len(train_cov)} = {train_match_n/len(train_cov)}")
print(f"[COVERAGE] TEST matches: {test_match_n}/{len(test_cov)} = {test_match_n/len(test_cov)}")

In [ ]:
hdb_unique = hdb.drop_duplicates(subset=["town","block"])

cols_to_add = ["latitude","longitude","max_floor","subzone","planning_area","region"]
cols_to_add = [c for c in cols_to_add if c in hdb_unique.columns]  # guard

train_df = train_df.merge(hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")
test_df = test_df.merge (hdb_unique[["town","block", *cols_to_add]], on=["town","block"], how="left")

In [ ]:
# Floor effects (relative position in block)
# Higher floors typically command a premium, and “how high” depends on the block’s max floor
# Relative floor and an “is high floor” flag. https://ideas.repec.org/a/taf/apeclt/v26y2019i6p436-439.html

for df in (train_df, test_df):
    df["rel_floor"] = (df["floor_mid"] / df["max_floor"]).replace([np.inf, -np.inf], np.nan)
    df["is_high_floor"] = (df["rel_floor"] >= 0.7).astype(int)

In [ ]:
# MRT Data
mrt = aux_dict["mrt"].copy()

mrt.columns = mrt.columns.str.lower()

for col in ["code","name", "status", "subzone", "planning_area", "region"]:
    if col in mrt.columns: mrt[col] = mrt[col].map(norm_key)

In [ ]:
print(mrt['status'].unique())
# Open and Planned MRT proximity features

from sklearn.neighbors import BallTree

mrt_open = mrt[mrt["status"] == "OPEN"].copy()
mrt_planned = mrt[mrt["status"] == "PLANNED"].copy()

R = 6371000.0
def to_rad(df): return np.deg2rad(df[["latitude", "longitude"]].to_numpy(dtype=float))

tree_open = BallTree(to_rad(mrt_open), metric="haversine")
tree_planned = BallTree(to_rad(mrt_planned), metric="haversine")

In [ ]:
def add_mrt_features(df, radii_open=(500,1000,2000), radii_plan=(500,1000,2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))

    # OPEN: nearest + counts
    if tree_open is not None and len(idx):
        dist, _ = tree_open.query(X, k=1)
        df.loc[idx, "mrt_open_nearest_m"] = dist[:,0] * R
        for r in radii_open:
            df.loc[idx, f"mrt_open_within_{r}m"] = tree_open.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_open_nearest_m"] = np.nan
        for r in radii_open: df[f"mrt_open_within_{r}m"] = 0

    # PLANNED: nearest + counts
    if tree_planned is not None and len(idx):
        dist, _ = tree_planned.query(X, k=1)
        df.loc[idx, "mrt_plan_nearest_m"] = dist[:,0] * R
        for r in radii_plan:
            df.loc[idx, f"mrt_plan_within_{r}m"] = tree_planned.query_radius(X, r/R, count_only=True)
    else:
        df["mrt_plan_nearest_m"] = np.nan
        for r in radii_plan: df[f"mrt_plan_within_{r}m"] = 0

    # Comparative signals
    df["mrt_any_nearest_m"] = np.nanmin(
        np.vstack([df["mrt_open_nearest_m"].to_numpy(dtype=float),
                   df["mrt_plan_nearest_m"].to_numpy(dtype=float)]),
        axis=0
    )
    df["mrt_plan_closer_than_open"] = (
        (df["mrt_plan_nearest_m"] < df["mrt_open_nearest_m"]).astype("Int64")
    )

    # fill NA counts for rows without coords
    for c in [col for col in df.columns if col.startswith(("mrt_open_within_","mrt_plan_within_"))]:
        df[c] = df[c].fillna(0).astype(int)

    return df

train_df = add_mrt_features(train_df)
test_df = add_mrt_features(test_df)

In [ ]:
# School Data
pri = aux_dict["pri"].copy()
sec = aux_dict["sec"].copy()

pri.columns = pri.columns.str.lower()
sec.columns = sec.columns.str.lower()

for col in ["name","street", "subzone", "planning_area", "region"]:
    if col in pri.columns: pri[col] = pri[col].map(norm_key)
    if col in sec.columns: sec[col] = sec[col].map(norm_key)

In [ ]:
# Distance to school
# MOE’s P1 registration use home-school distance categories of <1 km, 1–2 km, >2 km for priority, so those should capture a policy-driven price signal.

def add_proximity_to_school(df, tree, prefix, radii=(1000, 2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R
    # counts within radii (meters)
    for r in radii:
        cnt = tree.query_radius(X, r / R, count_only=True)
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
    # fill rows without coords
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
    return df

In [ ]:
# Primary schools
tree_pri = BallTree(to_rad(pri), metric="haversine")
train_df = add_proximity_to_school(train_df, tree_pri, "pri", radii=(1000, 2000))
test_df = add_proximity_to_school(test_df, tree_pri, "pri", radii=(1000, 2000))

# Secondary schools
tree_sec = BallTree(to_rad(sec), metric="haversine")
train_df = add_proximity_to_school(train_df, tree_sec, "sec", radii=(1000, 2000))
test_df = add_proximity_to_school(test_df, tree_sec, "sec", radii=(1000, 2000))

In [ ]:
mall = aux_dict["malls"].copy()
mall.columns = mall.columns.str.lower()
for col in ["name", "street", "postal_code", "latitude", "longitude", "subzone", "planning_area", "region"]:
    if col in pri.columns: pri[col] = pri[col].map(norm_key)

In [ ]:
def add_proximity_to_mall(df, tree, prefix, radii=(1000, 2000)):
    idx = df[["latitude","longitude"]].dropna().index
    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R
    # counts within radii (meters)
    for r in radii:
        cnt = tree.query_radius(X, r / R, count_only=True)
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
    # fill rows without coords
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
    return df

In [ ]:
tree_mall = BallTree(to_rad(mall), metric="haversine")
train_df = add_proximity_to_mall(train_df, tree_mall, "mall", radii=(1000, 2000))
test_df = add_proximity_to_mall(test_df, tree_mall, "mall", radii=(1000, 2000))

In [ ]:
hawker = aux_dict["hawkers"].copy()
hawker.columns = hawker.columns.str.lower()

In [ ]:
def add_proximity_to_hawker(df, tree, prefix, radii=(500, 1000)):
    idx = df[["latitude","longitude"]].dropna().index
    # nothing to do if no points with coords
    if len(idx) == 0:
        # ensure columns exist for downstream code
        for r in radii:
            df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
            df[f"{prefix}_stalls_within_{r}m"] = df.get(f"{prefix}_stalls_within_{r}m", 0).fillna(0).astype(int)
        df[f"{prefix}_nearest_m"] = np.nan
        return df

    X = np.deg2rad(df.loc[idx, ["latitude","longitude"]].to_numpy(dtype=float))
    # nearest distance (meters)
    dist, _ = tree.query(X, k=1)
    df.loc[idx, f"{prefix}_nearest_m"] = dist[:,0] * R

    # counts within radii (meters) and sum of stalls within radii
    stalls_col = 'number_of_stalls'
    for r in radii:
        inds = tree.query_radius(X, r / R)
        cnt = np.array([len(a) for a in inds])
        df.loc[idx, f"{prefix}_within_{r}m"] = cnt
        stalls_sum = np.array([hawker.iloc[a][stalls_col].sum() if len(a) > 0 else 0 for a in inds])
        df.loc[idx, f"{prefix}_stalls_within_{r}m"] = stalls_sum

    # fill rows without coords and ensure integer types for counts
    for r in radii:
        df[f"{prefix}_within_{r}m"] = df.get(f"{prefix}_within_{r}m", 0).fillna(0).astype(int)
        df[f"{prefix}_stalls_within_{r}m"] = df.get(f"{prefix}_stalls_within_{r}m", 0).fillna(0).astype(int)
    return df

In [ ]:
tree_hawker = BallTree(to_rad(hawker), metric="haversine")
train_df = add_proximity_to_hawker(train_df, tree_hawker, "hawker", radii=(500, 1000))
test_df = add_proximity_to_hawker(test_df, tree_hawker, "hawker", radii=(500, 1000))

In [ ]:
train_df.groupby("hawker_within_500m").size()

### Feature Engineering: Flat Age in Days

**Objective:** Create a precise age metric for HDB flats to capture depreciation effects.

This feature calculates the exact age of each flat in days from the lease commencement date to the sale date. Age is a crucial factor in property valuation as:
- Older flats typically have lower resale values
- Age affects remaining lease duration
- Newer flats may have modern amenities and better conditions

**Formula:** `flat_age_days = sale_date - lease_commence_date`

In [ ]:
def add_flat_age_days(train_df, test_df):

    train_df['lease_commence_dt'] = pd.to_datetime(train_df['lease_commence_date'].astype(str) + '-01-01')
    test_df['lease_commence_dt'] = pd.to_datetime(test_df['lease_commence_date'].astype(str) + '-01-01')

    train_df['flat_age_days'] = (train_df['month'] - train_df['lease_commence_dt']).dt.days
    test_df['flat_age_days'] = (test_df['month'] - test_df['lease_commence_dt']).dt.days

    train_df = train_df.drop(columns=['lease_commence_dt'])
    test_df = test_df.drop(columns=['lease_commence_dt'])

    return train_df, test_df

train_df, test_df = add_flat_age_days(train_df, test_df)
train_df

In [ ]:
# One-hot for coarse region, and target/mean encoding for higher-cardinality planning_area/subzone

# Region one-hot
region_dum = pd.get_dummies(train_df["region"], prefix="region", dummy_na=False)
train_df = pd.concat([train_df, region_dum], axis=1)
test_df = pd.concat([test_df, pd.get_dummies(test_df["region"], prefix="region", dummy_na=False)], axis=1)
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)  # align after dummies

# KFold mean encoding for planning_area
from sklearn.model_selection import KFold
def kfold_target_mean_encode(tr, te, col, y="resale_price", n_splits=5, seed=777):
    tr = tr.copy(); te = te.copy()
    global_mean = tr[y].mean()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)
    tr_enc = pd.Series(index=tr.index, dtype=float)
    for tr_idx, val_idx in kf.split(tr):
        m = tr.iloc[tr_idx].groupby(col)[y].mean()
        tr_enc.iloc[val_idx] = tr.iloc[val_idx][col].map(m).fillna(global_mean)
    te_enc = te[col].map(tr.groupby(col)[y].mean()).fillna(global_mean)
    tr[f"{col}_te"] = tr_enc
    te[f"{col}_te"] = te_enc
    return tr, te

train_df, test_df = kfold_target_mean_encode(train_df, test_df, "planning_area")
train_df, test_df = kfold_target_mean_encode(train_df, test_df, "subzone")

In [ ]:
# Improving upon flat-type price hierarchy

# Hedonic housing interaction
for df in (train_df, test_df):
    df["area_centered"] = df["floor_area_sqm"] - train_df["floor_area_sqm"].mean()
    df["exec_x_area"] = df["is_exec"] * df["area_centered"]
    df["multigen_x_area"] = df["is_multigen"] * df["area_centered"]

# coarse one-hot for model (limit top N categories to avoid sparsity)
top_models = train_df["flat_model"].value_counts().head(8).index
for df in (train_df, test_df):
    df["flat_model_top"] = df["flat_model"].where(df["flat_model"].isin(top_models), "OTHER")
model_dum = pd.get_dummies(train_df["flat_model_top"], prefix="model")
train_df = pd.concat([train_df, model_dum], axis=1)
test_df  = pd.concat([test_df, pd.get_dummies(test_df["flat_model_top"], prefix="model")], axis=1)
test_df = test_df.reindex(columns=train_df.columns, fill_value=0)


In [ ]:
print("="*70)
print("DATASET SHAPE AND STRUCTURE")
print("="*70)
print(f"Training set shape: {train_df.shape}")
print(f"Test set shape: {test_df.shape}")
print(f"\nNumber of features: {train_df.shape[1] - 1}")  # -1 for target variable
print(f"Number of training samples: {train_df.shape[0]:,}")
print(f"Number of test samples: {test_df.shape[0]:,}")

print("\n" + "="*70)
print("DATA TYPES")
print("="*70)
print(train_df.dtypes.value_counts())

print("\n" + "="*70)
print("MISSING VALUES")
print("="*70)
missing = train_df.isnull().sum()
if missing.sum() == 0:
    print("No missing values in training data!")
else:
    print(missing[missing > 0])

print("\n" + "="*70)
print("TARGET VARIABLE: resale_price")
print("="*70)
print(train_df['resale_price'].describe())
print(f"\nPrice Range: ${train_df['resale_price'].min():,.0f} - ${train_df['resale_price'].max():,.0f}")
print(f"Median Price: ${train_df['resale_price'].median():,.0f}")
print(f"Mean Price: ${train_df['resale_price'].mean():,.0f}")
print(f"Std Dev: ${train_df['resale_price'].std():,.0f}")

### 2. Distribution of Resale Prices

**Analysis:** Understanding the distribution shape helps determine appropriate preprocessing and modeling strategies.

**Key Observations:**
- **Right-skewed distribution:** Indicates presence of high-value outliers (premium properties)
- **Log transformation benefits:** Normalizes the distribution for better model performance
- **Central tendency measures:** Mean vs. median comparison reveals skewness impact

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Distribution of resale prices
axes[0].hist(train_df['resale_price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Distribution of Resale Prices', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Resale Price ($)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].axvline(train_df['resale_price'].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: ${train_df["resale_price"].mean():,.0f}')
axes[0].axvline(train_df['resale_price'].median(), color='green', linestyle='--', linewidth=2, label=f'Median: ${train_df["resale_price"].median():,.0f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Log-transformed distribution
axes[1].hist(np.log1p(train_df['resale_price']), bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_title('Log-Transformed Distribution of Resale Prices', fontsize=14, fontweight='bold')
axes[1].set_xlabel('log(Resale Price)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insight: Log transformation makes the distribution more Gaussian, which helps with modeling.")

### 3. Correlation Analysis - Numerical Features

**Objective:** Identify the strongest predictors of resale price through correlation analysis.

**What to look for:**
- **Strong positive correlations (r > 0.5):** Primary price drivers
- **Weak correlations (|r| < 0.2):** Features that may have limited predictive power
- **Negative correlations:** Features inversely related to price (e.g., distance to amenities)

**Note:** Correlation measures linear relationships only. Tree-based models can capture non-linear patterns.

In [ ]:
# Select key numerical features for correlation analysis
key_features = [
    'resale_price', 'floor_area_sqm', 'lease_commence_date', 
    'storey_range', 'sale_year', 'sale_month'
]

# Get all numerical columns
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()

# Calculate correlation with target
correlations = train_df[numerical_cols].corr()['resale_price'].sort_values(ascending=False)

print("="*70)
print("TOP 15 FEATURES MOST CORRELATED WITH RESALE PRICE")
print("="*70)
print(correlations.head(15))

print("\n" + "="*70)
print("TOP 10 FEATURES LEAST CORRELATED (MOST NEGATIVE) WITH RESALE PRICE")
print("="*70)
print(correlations.tail(10))

# Visualize top correlations
fig, ax = plt.subplots(figsize=(10, 8))
top_corr = correlations.head(20).sort_values()
top_corr.plot(kind='barh', ax=ax, color='steelblue', edgecolor='black')
ax.set_title('Top 20 Features Correlated with Resale Price', fontsize=14, fontweight='bold')
ax.set_xlabel('Correlation Coefficient', fontsize=12)
ax.axvline(x=0, color='red', linestyle='--', linewidth=1)
ax.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### 4. Correlation Heatmap - Key Features

**Purpose:** Visualize relationships between the most important features and identify multicollinearity.

**Interpretation Guide:**
- **Dark red (close to 1):** Strong positive correlation
- **Dark blue (close to -1):** Strong negative correlation
- **Light colors (close to 0):** Weak/no correlation
- **Off-diagonal patterns:** Feature interactions and multicollinearity

**Multicollinearity concerns:**
- High correlation between predictors (e.g., floor_area and bedrooms)
- May affect linear models but less critical for tree-based methods

In [ ]:
# Select most important features for heatmap
important_features = [
    'resale_price', 'floor_area_sqm', 'lease_commence_date', 
    'sale_year', 'flat_type_price_hier', 'bedrooms_est',
    'floor_mid', 'lease_left', 'month_index'
]

# Filter features that exist in the dataframe
available_features = [f for f in important_features if f in train_df.columns]

if len(available_features) > 1:
    # Calculate correlation matrix
    corr_matrix = train_df[available_features].corr()
    
    # Create heatmap
    plt.figure(figsize=(12, 10))
    import seaborn as sns
    sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
                square=True, linewidths=1, cbar_kws={"shrink": 0.8})
    plt.title('Correlation Heatmap of Key Features', fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
    print("\nKey Insights from Correlation Analysis:")
    print("1. floor_area_sqm likely has strong positive correlation with price")
    print("2. lease_left (remaining lease) likely correlates with price")
    print("3. flat_type_price_hier captures flat type effect on price")
    print("4. Temporal features (sale_year, month_index) may show market trends")
else:
    print("Not enough features available for correlation heatmap.")

### 5. Categorical Features Analysis

**Analysis Focus:** Examine how categorical variables (flat type, location) influence resale prices.

**Key Questions:**
1. **Flat Type Distribution:** Which flat types are most common in the dataset?
2. **Price Hierarchy:** Do larger flat types command premium prices?
3. **Location Effects:** How does geographic location (town) impact resale values?
4. **Market Concentration:** Are transactions concentrated in specific areas?

**Encoding Strategy:**
- **Low cardinality (flat_type):** One-hot encoding
- **High cardinality (town, planning_area):** Target/mean encoding to prevent feature explosion

In [ ]:
# Analyze flat type distribution and average prices
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Flat type distribution
flat_type_counts = train_df['flat_type'].value_counts()
axes[0, 0].barh(flat_type_counts.index, flat_type_counts.values, color='skyblue', edgecolor='black')
axes[0, 0].set_xlabel('Count', fontsize=12)
axes[0, 0].set_title('Distribution of Flat Types', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3, axis='x')

# 2. Average price by flat type
avg_price_by_type = train_df.groupby('flat_type')['resale_price'].mean().sort_values()
axes[0, 1].barh(avg_price_by_type.index, avg_price_by_type.values, color='coral', edgecolor='black')
axes[0, 1].set_xlabel('Average Resale Price ($)', fontsize=12)
axes[0, 1].set_title('Average Resale Price by Flat Type', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3, axis='x')

# 3. Town distribution (top 15)
town_counts = train_df['town'].value_counts().head(15)
axes[1, 0].barh(town_counts.index, town_counts.values, color='lightgreen', edgecolor='black')
axes[1, 0].set_xlabel('Count', fontsize=12)
axes[1, 0].set_title('Top 15 Towns by Number of Transactions', fontsize=14, fontweight='bold')
axes[1, 0].grid(alpha=0.3, axis='x')

# 4. Average price by town (top 15)
avg_price_by_town = train_df.groupby('town')['resale_price'].mean().sort_values(ascending=False).head(15)
axes[1, 1].barh(avg_price_by_town.index, avg_price_by_town.values, color='gold', edgecolor='black')
axes[1, 1].set_xlabel('Average Resale Price ($)', fontsize=12)
axes[1, 1].set_title('Top 15 Towns by Average Resale Price', fontsize=14, fontweight='bold')
axes[1, 1].grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print(f"1. Most common flat type: {flat_type_counts.index[0]}")
print(f"2. Highest average price flat type: {avg_price_by_type.index[-1]} (${avg_price_by_type.values[-1]:,.0f})")
print(f"3. Most transactions in town: {town_counts.index[0]}")
print(f"4. Highest average price town: {avg_price_by_town.index[0]} (${avg_price_by_town.values[0]:,.0f})")

### 6. Temporal Trends - Price Evolution Over Time

**Objective:** Analyze how HDB resale prices have changed over time and identify market trends.

**Analysis Components:**
1. **Year-over-year price trends:** Identify appreciation/depreciation patterns
2. **Transaction volume trends:** Market activity levels over time
3. **Seasonal effects:** Price variations by quarter/month (if any)

**Modeling Implications:**
- Increasing prices --> Include temporal features (year, month_index)
- Market cycles --> Polynomial/interaction terms for sale year
- Volume changes --> May indicate market sentiment shifts

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 1. Average price by year
yearly_avg = train_df.groupby('sale_year')['resale_price'].agg(['mean', 'median', 'count'])
axes[0].plot(yearly_avg.index, yearly_avg['mean'], marker='o', linewidth=2, label='Mean Price', color='blue')
axes[0].plot(yearly_avg.index, yearly_avg['median'], marker='s', linewidth=2, label='Median Price', color='green')
axes[0].set_xlabel('Year', fontsize=12)
axes[0].set_ylabel('Resale Price ($)', fontsize=12)
axes[0].set_title('Average Resale Price Trend Over Years', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# 2. Transaction volume by year
axes[1].bar(yearly_avg.index, yearly_avg['count'], color='orange', edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Year', fontsize=12)
axes[1].set_ylabel('Number of Transactions', fontsize=12)
axes[1].set_title('Transaction Volume Over Years', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("Temporal Analysis:")
print(yearly_avg)
print(f"\nPrice increase from {yearly_avg.index.min()} to {yearly_avg.index.max()}: "
      f"${yearly_avg['mean'].iloc[-1] - yearly_avg['mean'].iloc[0]:,.0f} "
      f"({((yearly_avg['mean'].iloc[-1] / yearly_avg['mean'].iloc[0]) - 1) * 100:.1f}%)")

### 7. Key Feature Relationships - Scatter Plots

**Purpose:** Visualize bivariate relationships between continuous features and resale price.

**Patterns to Identify:**
1. **Linear relationships:** Constant rate of change (e.g., floor area vs. price)
2. **Non-linear patterns:** Diminishing/accelerating returns
3. **Heteroscedasticity:** Variance changes with feature values
4. **Outliers and anomalies:** Unusual data points requiring investigation

**Model Selection Insights:**
- Linear patterns → Linear regression may suffice
- Non-linear patterns → Tree-based models or polynomial features needed
- Complex interactions → Gradient boosting recommended

In [ ]:
# Scatter plots to visualize relationships
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Sample data for faster plotting
sample_df = train_df.sample(n=min(5000, len(train_df)), random_state=777)

# 1. Floor area vs Price
axes[0, 0].scatter(sample_df['floor_area_sqm'], sample_df['resale_price'], alpha=0.3, s=10)
axes[0, 0].set_xlabel('Floor Area (sqm)', fontsize=12)
axes[0, 0].set_ylabel('Resale Price ($)', fontsize=12)
axes[0, 0].set_title('Floor Area vs Resale Price', fontsize=14, fontweight='bold')
axes[0, 0].grid(alpha=0.3)

# 2. Lease commence year vs Price
axes[0, 1].scatter(sample_df['lease_commence_date'], sample_df['resale_price'], alpha=0.3, s=10, color='green')
axes[0, 1].set_xlabel('Lease Commence Year', fontsize=12)
axes[0, 1].set_ylabel('Resale Price ($)', fontsize=12)
axes[0, 1].set_title('Lease Commence Year vs Resale Price', fontsize=14, fontweight='bold')
axes[0, 1].grid(alpha=0.3)

# 3. Floor level vs Price (if available)
if 'floor_mid' in sample_df.columns:
    axes[1, 0].scatter(sample_df['floor_mid'], sample_df['resale_price'], alpha=0.3, s=10, color='orange')
    axes[1, 0].set_xlabel('Floor Level (mid)', fontsize=12)
    axes[1, 0].set_ylabel('Resale Price ($)', fontsize=12)
    axes[1, 0].set_title('Floor Level vs Resale Price', fontsize=14, fontweight='bold')
    axes[1, 0].grid(alpha=0.3)

# 4. Remaining lease vs Price (if available)
if 'lease_left' in sample_df.columns:
    axes[1, 1].scatter(sample_df['lease_left'], sample_df['resale_price'], alpha=0.3, s=10, color='red')
    axes[1, 1].set_xlabel('Remaining Lease (years)', fontsize=12)
    axes[1, 1].set_ylabel('Resale Price ($)', fontsize=12)
    axes[1, 1].set_title('Remaining Lease vs Resale Price', fontsize=14, fontweight='bold')
    axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Relationships Observed:")
print("1. Floor area shows strong positive correlation with price")
print("2. Newer flats (higher lease commence year) tend to have higher prices")
print("3. Higher floors may command premium prices")
print("4. Remaining lease length affects property valuation")

### 8. EDA Summary and Key Findings for Report

**Purpose:** Synthesize all EDA findings into actionable insights for model development.

This summary section provides:
- **Quantitative findings:** Key statistics and metrics
- **Qualitative insights:** Patterns and relationships discovered
- **Feature importance:** Which variables matter most
- **Preprocessing recommendations:** Data transformations needed
- **Model selection rationale:** Why certain algorithms are preferred

**Use this summary for:**
- Report writing (Results and Discussion sections)
- Model selection justification
- Feature engineering decisions
- Performance expectations

In [ ]:
print("="*80)
print("EXPLORATORY DATA ANALYSIS SUMMARY - KEY FINDINGS FOR REPORT")
print("="*80)

print("\n  1. DATASET CHARACTERISTICS:")
print(f"   - Total training samples: {train_df.shape[0]:,}")
print(f"   - Number of features: {train_df.shape[1] - 1}")
print(f"   - No missing values in dataset")
print(f"   - Price range: ${train_df['resale_price'].min():,.0f} - ${train_df['resale_price'].max():,.0f}")
print(f"   - Median price: ${train_df['resale_price'].median():,.0f}")

print("\n  2. STRONGEST PREDICTORS (Top Correlations):")
numerical_cols = train_df.select_dtypes(include=[np.number]).columns.tolist()
correlations = train_df[numerical_cols].corr()['resale_price'].sort_values(ascending=False)
print(f"   - Floor area (correlation: {correlations.get('floor_area_sqm', 'N/A'):.3f})")
print(f"   - Flat type hierarchy")
print(f"   - Remaining lease years")
print(f"   - Location (town, planning area)")
print(f"   - Floor level")

print("\n  3. TEMPORAL TRENDS:")
yearly_avg = train_df.groupby('sale_year')['resale_price'].mean()
print(f"   - Prices show {'upward' if yearly_avg.iloc[-1] > yearly_avg.iloc[0] else 'downward'} trend over time")
print(f"   - Market shows temporal patterns requiring time-based features")

print("\n  4. CATEGORICAL FEATURES INSIGHTS:")
print(f"   - {len(train_df['town'].unique())} unique towns")
print(f"   - {len(train_df['flat_type'].unique())} flat types")
print(f"   - Significant price variation across locations (high cardinality)")
print(f"   - Flat type strongly influences price")

print("\n 5. FEATURE INTERACTIONS:")
print("   - Floor area x Flat type: Larger flats of premium types command higher prices")
print("   - Location x Time: Different areas show different appreciation rates")
print("   - Lease remaining x Sale year: Depreciation effects on older flats")

print("\n 6. DATA PREPROCESSING REQUIREMENTS:")
print("   - Log transformation beneficial for price distribution")
print("   - One-hot encoding needed for low-cardinality categorical features")
print("   - Target/mean encoding for high-cardinality features (town, planning area)")
print("   - Feature engineering: temporal, proximity, and interaction features")

print("\n 7. MODELING IMPLICATIONS:")
print("   - Non-linear relationships suggest tree-based models will perform well")
print("   - High feature dimensionality after encoding → gradient boosting preferred")
print("   - Temporal patterns --> include time-based features")
print("   - Location importance --> proximity features to amenities valuable")

print("\n 8. WHY POLYNOMIAL REGRESSION IS SUBOPTIMAL:")
print("   - High-dimensional feature space (100+ features after encoding)")
print("   - Polynomial expansion creates 1000s of features--> overfitting risk")
print("   - Non-linear patterns better captured by tree-based models")
print("   - Computational cost: O(n^2) for degree 2, O(n^3) for degree 3")
print("   - XGBoost/LightGBM capture interactions automatically without explosion")

print("\n" + "="*80)
print("CONCLUSION: Gradient Boosting (XGBoost/LightGBM) is optimal for this problem")
print("="*80)

---
## 🤖 Baseline Models - Performance Benchmarking

This section implements various regression algorithms to establish performance baselines and compare model effectiveness.

**Models Evaluated:**
1. Linear Regression (simplest baseline)
2. Polynomial Regression (degree 2, with optimization)
3. Decision Tree Regression
4. Bagging Regressor
5. Random Forest Regressor
6. **XGBoost (best performer)**

**Evaluation Metric:** Root Mean Squared Error (RMSE) on validation set (20% holdout)

### Model 1: Linear Regression (Baseline)

**Algorithm:** Ordinary Least Squares (OLS) Linear Regression

**Characteristics:**
- Simplest interpretable model
- Assumes linear relationships between features and target
- Fast training and prediction
- Works well with properly scaled features

**Expected Performance:** ~55,000 RMSE (highest error among models)

**Limitations:**
- Cannot capture non-linear relationships
- Sensitive to outliers
- Assumes feature independence (multicollinearity issues)

In [ ]:
target = 'resale_price'

# Define columns that should NOT be used as features
# block and street are in lat lon
# month and lease_commence_date are in flat_age_days
drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']


numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

# Create a validation split (80% train, 20% validation)
X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=777
)

model_pipeline.fit(X_train_split, y_train_split)
val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

# retraining on all data
model_pipeline.fit(X, y)

# Save the entire pipeline (preprocessor + model) to a file
model_filename = 'linear_regression.joblib'
joblib.dump(model_pipeline, model_filename)

# Validation RMSE: 55385.9387
# ['linear_regression.joblib']


### Model 2: Polynomial Regression (Degree 2) - Optimized

**Algorithm:** Ridge Regression with Polynomial Features (degree=2, interaction_only=True)

**Optimization Strategies:**
1. **Feature Selection:** SelectKBest limits to top 30 numerical features
2. **Interaction Terms Only:** Avoids feature explosion (no squared terms)
3. **Sparse Matrices:** Memory-efficient representation
4. **Ridge Regularization:** Handles multicollinearity better than OLS
5. **LSQR Solver:** Compatible with all scipy versions

**Why interaction_only=True?**
- Full polynomial expansion creates O(n^2) features
- With 100+ features, degree 2 creates 5,000+ features -> overfitting
- Interaction terms capture relationships without explosion

**Trade-off:** Better than linear but slower training. Still inferior to tree-based methods.

In [ ]:
# Optimized Polynomial Regression (CPU) - using feature selection and sparse matrices
target = 'resale_price'
drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

# Strategy 1: Reduce features before polynomial expansion
# Select only the most important numerical features to reduce computational burden
from sklearn.feature_selection import SelectKBest, f_regression

print(f"Original numerical features: {len(numerical_features)}")

# Use a subset of features for faster polynomial expansion
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('selector', SelectKBest(f_regression, k=min(30, len(numerical_features))))  # Limit to top 30 features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True))  # Keep sparse
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='drop',
    sparse_threshold=0.3  # Use sparse matrices when beneficial
)

# Strategy 2: Use interaction_only=True to reduce feature explosion
# This creates only interaction terms, not powers (much fewer features)
from sklearn.linear_model import Ridge  # Ridge handles multicollinearity better

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)),  # Much faster!
    ('model', Ridge(alpha=1.0))  # Ridge is more stable than OLS for many features
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=777
)

print("Training optimized polynomial model (interaction_only=True)...")
print("This should be much faster than full polynomial expansion!")

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

# Retrain on full data
print("Retraining on full dataset...")
model_pipeline.fit(X, y)

model_filename = 'polynomial_regression_optimized.joblib'
joblib.dump(model_pipeline, model_filename)

print(f"Model saved to {model_filename}")
print(f"\nOptimization strategies used:")
print("  1. Feature selection: Reduced to top 30 numerical features")
print("  2. interaction_only=True: Only interaction terms, no powers")
print("  3. Sparse matrices: Memory efficient representation")
print("  4. Ridge regression: Better regularization than OLS")

### Model 3: Decision Tree Regression

**Algorithm:** Single Decision Tree (CART)

**Characteristics:**
- Non-linear model that splits feature space recursively
- Can capture complex relationships and interactions automatically
- No feature scaling required
- Prone to overfitting without regularization

**Expected Performance:** ~42,000 RMSE

**Limitations:**
- High variance (unstable predictions)
- Overfits easily on training data
- Not ensemble method -> lower accuracy than Random Forest

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=777)) # Apply Decision Tree model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=777
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'decision_tree.joblib'
joblib.dump(model_pipeline, model_filename)

# Validation RMSE: 42634.5061
# ['decision_tree.joblib']

### Model 4: Bagging Regressor

**Algorithm:** Bootstrap Aggregating (Bagging) with Decision Trees

**How it Works:**
1. Create multiple bootstrap samples from training data
2. Train independent decision trees on each sample
3. Average predictions from all trees

**Advantages:**
- Reduces variance compared to single decision tree
- Parallel training of trees (faster than boosting)
- Less prone to overfitting than single tree

**Expected Performance:** ~31,500 RMSE

**Limitation:** Trees trained independently (doesn't learn from previous errors like boosting)

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', BaggingRegressor(random_state=777)) # Apply Bagging Regression model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=777
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'bagging.joblib'
joblib.dump(model_pipeline, model_filename)

# Validation RMSE: 31510.9685
# ['bagging.joblib']

### Model 5: Random Forest Regression

**Algorithm:** Ensemble of Decision Trees with Random Feature Selection

**How it Improves on Bagging:**
1. Bootstrap sampling (like bagging)
2. **Random feature subset** at each split → decorrelates trees
3. Averages predictions from many diverse trees

**Advantages:**
- Lower variance than bagging
- Feature importance scores
- Robust to outliers and missing values
- Works well out-of-the-box

**Expected Performance:** ~29,800 RMSE

**Why Still Inferior to XGBoost?**
- Random Forest: Trees built independently
- XGBoost: Trees built sequentially, learning from errors → better accuracy

In [ ]:
target = 'resale_price'

drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

numerical_features = [
    col for col in train_df.select_dtypes(include=np.number).columns
    if col not in drop_cols
]

categorical_features = [
    col for col in train_df.select_dtypes(include=['object', 'category']).columns
    if col not in drop_cols
]

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Handle missing values
    ('scaler', StandardScaler())                    # Normalize data
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Handle missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=777)) # Apply Random Forest Regression model
])

X = train_df.drop(columns=drop_cols)
y = train_df[target]

X_train_split, X_val, y_train_split, y_val = train_test_split(
    X, y, test_size=0.2, random_state=777
)

model_pipeline.fit(X_train_split, y_train_split)

val_predictions = model_pipeline.predict(X_val)

# Calculate RMSE
validation_mse = mean_squared_error(y_val, val_predictions)
validation_rmse = np.sqrt(validation_mse)

print(f"Validation RMSE: {validation_rmse:.4f}")

model_pipeline.fit(X, y)

model_filename = 'random_forest.joblib'
joblib.dump(model_pipeline, model_filename)

# Validation RMSE: 29868.7537
# ['random_forest.joblib']

### Model 6: XGBoost (Extreme Gradient Boosting) - Best Model 

**Algorithm:** Gradient Boosting with Advanced Regularization

**Why XGBoost is Superior:**
1. **Sequential learning:** Each tree corrects errors of previous trees
2. **Regularization:** L1 (alpha) and L2 (lambda) prevent overfitting
3. **Advanced tree pruning:** Max depth, min child weight, gamma
4. **Column/row subsampling:** Reduces overfitting and training time
5. **Built-in cross-validation:** Early stopping based on validation performance

**Hyperparameter Tuning Strategy:**
- **40 random search trials** (good balance of exploration vs. time)
- **Log-transformed target:** More Gaussian residuals, better handling of outliers
- **Early stopping (200 rounds):** Prevents overtraining
- **Validation RMSE:** ~25,400 (best among all models)

**Implementation Details:**
- GPU acceleration enabled (if available)
- Saves best model automatically to `models/xgb_best/`
- Returns preprocessor + model for easy deployment

In [ ]:
def train_xgboost_model(
    train_df,
    target: str = "resale_price",
    drop_cols: Optional[List[str]] = None,
    numerical_features: Optional[List[str]] = None,
    categorical_features: Optional[List[str]] = None,
    val_size: float = 0.2,
    random_state: int = 777,
    use_gpu: bool = True,
    n_trials: int = 30,
    early_stopping_rounds: int = 200,
    max_boost_rounds: int = 5000,
    save_dir: Optional[str] = None,
) -> Dict[str, Any]:
    
    # Initialize random number generator for hyperparameter sampling
    rng = np.random.default_rng(random_state)


    # Column prep
    drop_cols = [target, 'month', 'block', 'street', 'lease_commence_date']

    X = train_df.drop(columns=drop_cols, errors="ignore")
    y = train_df[target].to_numpy()

    if numerical_features is None:
        numerical_features = list(X.select_dtypes(include=np.number).columns)
    if categorical_features is None:
        categorical_features = list(X.select_dtypes(include=["object", "category"]).columns)

    if not numerical_features and not categorical_features:
        raise ValueError("No features found. Check drop_cols and dtypes.")

    # Preprocessor
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", Pipeline([("imp", SimpleImputer(strategy="median"))]), numerical_features),
            ("cat", Pipeline([("imp", SimpleImputer(strategy="most_frequent")), ("ohe", ohe)]), categorical_features),
        ],
        remainder="drop",
    )

    X_train, X_val, y_train_raw, y_val_raw = train_test_split(
        X, y, test_size=val_size, random_state=random_state
    )

    X_train_t = preprocessor.fit_transform(X_train)
    X_val_t = preprocessor.transform(X_val)

    # Log-transform target for more Gaussian residuals, this should help with outliers
    y_train = np.log1p(y_train_raw)
    y_val = np.log1p(y_val_raw)

    def make_params(overrides: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        params = {
            "objective": "reg:squarederror",
            "eval_metric": "rmse",
            "max_depth": 8,
            "min_child_weight": 2.0,
            "subsample": 0.8,
            "colsample_bytree": 0.8,
            "alpha": 0.0, # L1
            "lambda": 1.0, # L2
            "gamma": 0.0,
            "grow_policy": "lossguide",
        }
        if use_gpu:
            params.update({"device": "cuda", "tree_method": "hist"})
        else:
            params.update({"tree_method": "hist"})
        if overrides:
            lr = overrides.pop("learning_rate", None)
            if lr is not None:
                params["eta"] = lr
            params.update(overrides)
        return params

    # Random sampling
    def sample_params() -> Dict[str, Any]:
        return {
            "learning_rate": 10 ** rng.uniform(-2.3, -0.7), # ~0.005..0.2
            "max_depth": int(rng.integers(4, 13)), # 4..12
            "min_child_weight": float(10 ** rng.uniform(-0.3, 1.0)), # ~0.5..10
            "subsample": float(rng.uniform(0.6, 1.0)),
            "colsample_bytree": float(rng.uniform(0.6, 1.0)),
            "alpha": float(10 ** rng.uniform(-3, 1)), # 0.001..10
            "lambda": float(10 ** rng.uniform(-2, 1)), # 0.01..10
            "gamma": float(10 ** rng.uniform(-3, 0)), # 0.001..1
            "max_leaves": int(rng.integers(16, 512)),
        }

    # Helpers to use "best trees"
    def best_ntrees(booster: xgb.Booster) -> int:
        if hasattr(booster, "best_iteration") and booster.best_iteration is not None:
            return int(booster.best_iteration) + 1
        if hasattr(booster, "best_ntree_limit") and booster.best_ntree_limit:
            return int(booster.best_ntree_limit)
        return max_boost_rounds

    def predict_best(booster: xgb.Booster, dmat: xgb.DMatrix) -> np.ndarray:
        nbest = best_ntrees(booster)
        try:
            return booster.predict(dmat, iteration_range=(0, nbest))
        except TypeError:
            return booster.predict(dmat, ntree_limit=nbest)

    dtrain = xgb.DMatrix(X_train_t, label=y_train)
    dval = xgb.DMatrix(X_val_t, label=y_val)

    # Hyperparameter search
    best: Dict[str, Any] = {"rmse": math.inf, "params": None, "best_iteration": None, "booster": None}

    pbar = tqdm(range(n_trials), unit="trial", desc="XGB hyperparam search")
    for _ in pbar:
        trial = sample_params()
        params = make_params(trial)
        booster = xgb.train(
            params=params,
            dtrain=dtrain,
            num_boost_round=max_boost_rounds,
            evals=[(dval, "val")],
            early_stopping_rounds=early_stopping_rounds,
            verbose_eval=False,
        )
        # score on original scale
        val_pred = np.expm1(predict_best(booster, dval))
        rmse = float(np.sqrt(mean_squared_error(y_val_raw, val_pred)))

        if rmse < best["rmse"]:
            best.update({
                "rmse": rmse,
                "params": params,
                "best_iteration": best_ntrees(booster) - 1,  # store 0-based
                "booster": booster,
            })
        pbar.set_postfix(rmse=f"{rmse:.2f}", best=f"{best['rmse']:.2f}")

    # Refit with best params
    X_full_t = preprocessor.transform(X)
    y_full = np.log1p(y)
    dfull = xgb.DMatrix(X_full_t, label=y_full)

    final_params = dict(best["params"])
    final_rounds = int(best["best_iteration"]) + 1

    bst_final = xgb.train(
        params=final_params,
        dtrain=dfull,
        num_boost_round=final_rounds,
        verbose_eval=False,
    )

    # Adapter for the prediction
    class BoosterAdapter:
        def __init__(self, booster: xgb.Booster, ntrees: int):
            self.booster = booster
            self.best_iteration = ntrees - 1
        def predict(self, Xmat) -> np.ndarray:
            dmat = xgb.DMatrix(Xmat)
            try:
                return self.booster.predict(dmat, iteration_range=(0, self.best_iteration + 1))
            except TypeError:
                return self.booster.predict(dmat, ntree_limit=self.best_iteration + 1)
        def __repr__(self) -> str:
            return f"BoosterAdapter(best_iteration={self.best_iteration})"

    model_adapter = BoosterAdapter(bst_final, final_rounds)

    if save_dir:
        path = Path(save_dir)
        path.mkdir(parents=True, exist_ok=True)
        joblib.dump(preprocessor, path / "xgb_model.joblib")
        bst_final.save_model(str(path / "xgb_model.json"))
        with (path / "best_summary.json").open("w") as f:
            json.dump(
                {
                    "val_rmse": best["rmse"],
                    "best_iteration": int(best["best_iteration"]),
                    "num_boost_round": final_rounds,
                    "best_params": {
                        k: final_params[k]
                        for k in [
                            "eta", "max_depth", "min_child_weight", "subsample",
                            "colsample_bytree", "alpha", "lambda", "gamma",
                            "grow_policy", *(["device"] if "device" in final_params else []),
                            "tree_method",
                        ]
                        if k in final_params
                    },
                },
                f,
                indent=2,
            )

    return {
        "preprocessor": preprocessor,
        "model": model_adapter,
        "best_params": final_params,
        "best_iteration": int(best["best_iteration"]),
        "val_rmse": float(best["rmse"]),
    }

In [ ]:
result = train_xgboost_model(
    train_df,
    target="resale_price",
    drop_cols=["resale_price", "month", "block", "street", "lease_commence_date"],
    save_dir="models/xgb_best",
    n_trials=40,
    use_gpu=True,
)
print(f"Best val RMSE: {result['val_rmse']:.2f}")

# Best val RMSE = 25434.09
# ['xgb_model.joblib']

---
## Model Deployment and Predictions

**Final Step:** Generate predictions on test set using the best-performing XGBoost model.

**Process:**
1. Load trained preprocessor and model
2. Transform test data using same preprocessing pipeline
3. Generate predictions (remember to inverse log-transform)
4. Create submission file in required format

**Output:** `submission.csv` ready for Kaggle submission

In [ ]:
loaded_model = joblib.load('C:\\Users\\GoCh661\\Downloads\\cs5228\\models\\xgb_best\\xgb_model.joblib') #please change based on your path
X_sub = test_df.drop(columns=["resale_price", "month", "block", "street", "lease_commence_date"], errors="ignore")
X_sub_t = result["preprocessor"].transform(X_sub)
final_predictions = np.expm1(result["model"].predict(X_sub_t))
results_df = pd.DataFrame({'Id': test_df.index,
                           'Predicted': final_predictions})
results_df.to_csv("submission.csv", index=False)